# Optimisation des Performances d’un Épurateur de Fumée (RTO) pour le traitement d’H2S

In [1]:
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt

## ETAPE 1 : Définition d'une composition représentative du gaz à l'entrée de l'épurateur (sortie de four cimenterie)

In [12]:
# Hypothèse : Propane richesse 1.5 à l'équilibre chimique (à adapter)
# données gri30
gas_sortiefour = ct.Solution('gri30.yaml')
# mélange frais à température et pression constante
gas_sortiefour.set_equivalence_ratio(3, 'C3H8', 'O2:0.21, N2:0.79')
gas_sortiefour.TP = 300, 101325
# composition gaz brulés
gas_sortiefour.equilibrate('HP')

# on affiche la température pour info et on enregistre dans une variable la masse volumique du gaz
gas_sortiefour()
#print('température sortie four',gas_sortiefour.T)
#rho_sortiefour = gas_sortiefour.density


  gri30:

       temperature   1080.8 K
          pressure   1.0132e+05 Pa
           density   0.23279 kg/m^3
  mean mol. weight   20.646 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -3.7822e+05       -7.8087e+06  J
   internal energy       -8.1348e+05       -1.6795e+07  J
           entropy             10885        2.2474e+05  J/K
    Gibbs function       -1.2143e+07       -2.5071e+08  J
 heat capacity c_p            1596.9             32970  J/K
 heat capacity c_v            1194.2             24656  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2          0.027745           0.28414           -18.961
                 H        4.3679e-10        8.9464e-09           -9.4803
                OH        1.5301e-12        1.8575e-12           -46.

## ETAPE 2 : Échange thermique à l'entrée de l'épurateur : Modification de la composition et température du gaz, déstockage de l'énergie des céramiques

In [13]:
# création d'un gaz fictif pour échanger l'énergie avec le gaz sortie de four (ça remplace les céramiques).
ceramique = ct.Solution('gri30.yaml')
ceramique.TPX = 1500.0, ct.one_atm, "O2:0.21, N2:0.79"
# création d'un réacteur contenant ce gaz 
reacteur_ceramique = ct.ConstPressureReactor(ceramique)
# création d'un réacteur contenant le gaz sortant du four et entrant dans l'échangeur
reacteur_gaz = ct.ConstPressureReactor(gas_sortiefour)
# création de la paroi entre les deux réacteurs
paroi = ct.Wall(reacteur_gaz, reacteur_ceramique, A=1.0, U=1000.0)
echangeur = ct.ReactorNet([reacteur_gaz, reacteur_ceramique])

time = 0.0
n_steps = 1000
for n in range(n_steps):
    time += 4.e-4
    echangeur.advance(time)
#    print(n, time, reacteur_gaz.thermo.T, reacteur_gaz.thermo.P, reacteur_gaz.volume, reacteur_ceramique.thermo.T,
#         reacteur_ceramique.thermo.P, reacteur_ceramique.volume)

#gas_sortiefour()
print('température réacteur gaz', reacteur_gaz.thermo.T, gas_sortiefour.T)

température réacteur gaz 1245.663419786708 1245.663419786708


## ETAPE 3 : Combustion dans l'épurateur : Modification de la composition et température du gaz, craquage du H2S

In [14]:
# création des réservoirs amonts et aval de la zone de mélange:
# syntax pour créer un réservoir :
# reservoir = ct.Reservoir(gas_reservoir)
# réservoir amont contenant gaz sortant du four
res_entree_comb= ct.Reservoir(gas_sortiefour)
rho_entree_comb= gas_sortiefour.density
# réservoir aval qu'on rempli initialement d'un gaz et qui va recevoir ce qui sort du mélangeur.
res_sortie_comb = ct.Reservoir(gas_sortiefour)
# on crée un réservoir contenant de l'air.
air=ct.Solution('gri30.yaml')
air.TPX = 300, 101325,'O2:0.21, N2:0.79'
res_air = ct.Reservoir(air)
# on enregistre dans une variable la masse volumique de l'air
rho_air= air.density

# création d'un réacteur/mélangeur rempli initialement d'un gaz.
# syntax pour créer un réservoir :
# reacteur = ct.IdealGasReactor(gaz_init)
combustor = ct.IdealGasReactor(gas_sortiefour)

# calcul des débits massiques entrant dans le réacteur et affichage:
dem_entree_comb=rho_entree_comb*0.5
print('débit massique entrant chambre de combustion',dem_entree_comb)
dem_air=rho_air*0.35
print('débit massique air',dem_air)

# définition des entrées dans le réacteur:
# syntax pour les entrées :
# entree = ct.MassFlowController(reservoir, reacteur, mdot=débit_massique)
mfc1 = ct.MassFlowController(res_entree_comb, combustor, mdot=dem_entree_comb)
mfc2 = ct.MassFlowController(res_air, combustor, mdot=dem_air)

# définition de la sortie du réacteur:
# syntax pour la sortie :
# sortie = ct.Valve(reacteur, reservoir, K=10.0) K est une constante de pertes de charges sans influence dans notre cas.
outlet = ct.Valve(combustor, res_sortie_comb, K=10.0)

# création du réseau de réacteur (nécessaire même pour un seul réacteur):
# syntax :
#sim = ct.ReactorNet([reacteur1, reacteur2])
sim = ct.ReactorNet([combustor])

# recherche de la solution stationnaire de l'état du réacteur:
sim.advance_to_steady_state()
# affichage résultat
#print(r1.thermo.report())
print('température après mélange air',combustor.thermo.T, gas_sortiefour.T)

débit massique entrant chambre de combustion 0.10099223040266274
débit massique air 0.41018962230387923
température après mélange air 1574.13605552209 1574.13605552209


## ETAPE 4 : Échange thermique à la sortie de l'épurateur : Modification de la composition et température du gaz, stockage de l'énergie dans les céramiques

In [ ]:
# a faire comme etape 2

## ETAPE HYPOTHETIQUE : Refroidissement par introduction d'eau liquide

In [15]:
#gas = ct.Solution('gri30.yaml')
#gas.TP = 1500.0, ct.one_atm
reactor_gas = ct.IdealGasReactor(gas_sortiefour)
water = ct.Water()
liquid_water_T = 300.0  # Température de l'eau en K
liquid_water_P = ct.one_atm  # Pression de l'eau en Pa
water.TP = liquid_water_T, liquid_water_P
#water()
print(f"Température du gaz : {gas_sortiefour.T} K")
print(f"Température de l'eau : {water.T} K")
reactor_liquid = ct.Reactor(water)
connection = ct.Wall(reactor_gas, reactor_liquid, A=0.1, U=100.0)
sim = ct.ReactorNet([reactor_gas, reactor_liquid])
#sim = ct.ReactorNet([reactor_gas, reservoir_liquid, heat_exchanger])
time_end = 100.0  # Temps final de la simulation en secondes
sim.advance(time_end)
#sim.advance_to_steady_state()
# Imprimer les résultats
print("Après mélange avec de l'eau liquide:")
print(f"Température du gaz : {gas_sortiefour.T} K")
print(f"Température de l'eau : {water.T} K")

Température du gaz : 1574.13605552209 K
Température de l'eau : 300.0 K
Après mélange avec de l'eau liquide:
Température du gaz : 304.8965085522349 K
Température de l'eau : 300.06212883683895 K


## Calcul sur plusieurs cycles des étapes précédentes